In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pandas as pd
import numpy as np

In [2]:
mousecsv_df = pd.read_csv('data/Pymaceuticals_data_Mouse_metadata.csv')
studycsv_df = pd.read_csv('data/Pymaceuticals_data_Study_results.csv')
studycsv_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [3]:
merge_df = pd.merge(mousecsv_df, studycsv_df, on='Mouse ID')
merge_df['Timepoint'].count() #Count original amount of Timestamp

#Align text to the left
def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
        [ dict(selector='th', props=[('text-align', 'left')] ) ])
    
    return left_aligned_df

In [4]:

clean_merge = merge_df.drop_duplicates(subset=['Mouse ID', 'Timepoint'],keep=False)
#duplicate_id = merge_df[merge_df.duplicated(subset=['Mouse ID', 'Timepoint'], keep='first')]*******
#clean_merge = clean_merge.drop(duplicate_id['Mouse ID'],axis=1)

#a = list(duplicate_id.index.values)#******
#clean_merge = clean_merge.drop([clean_merge.index[908]])
#clean_merge = clean_merge.drop([clean_merge.index[909], clean_merge.index[910], clean_merge.index[912], clean_merge.index[914], clean_merge.index[916]])
#pos = [908,910,912,914,916]
#clean_merge = clean_merge.drop([clean_merge.index[pos]])
#extra = list(duplicate_id['Mouse ID'])*******


#clean_merge = x for x in pos [clean_merge.drop([clean_merge.index[x]])]
#df_1_2 = df1.merge(df2, on="id", how="left", indicator=True)

#clean_merge = clean_merge.loc[clean_merge.index.difference(duplicate_id.index), ]

#clean_merge = clean_merge.drop([duplicate_id['Mouse ID'].index.values.astype(int)]) 
#clean_merge = clean_merge[(clean_merge['Mouse ID'] != duplicate_id['Mouse ID'])]
clean_merge = clean_merge[(clean_merge['Mouse ID'] != 'g989')]

clean_merge['Timepoint'].count() #Count Timestamp without duplicates
#print(extra)
#duplicate_id.head()
#print(duplicate_id.index.values)

1880

In [51]:
#Grouped data by Drug Regimen
group_stats = merge_df.groupby(['Drug Regimen'])

#Summary statistics for each drug regimen
general_stats = group_stats.describe()

#Median of the Tumor Volume for each Drug Regimen
median_stats = group_stats.median()

#Variance of the Tumor Volume for each Drug Regimen
variance_stats = group_stats.var()

#SEM of the Tumor Volume for each Drug Regimen
sem_stats = group_stats.sem()

#Created a Summary Statistics Data Frame
sem_stats_df = pd.DataFrame({'Mean':general_stats['Tumor Volume (mm3)']['mean'],
                             'Median':median_stats['Tumor Volume (mm3)'],
                             'Variance':variance_stats['Tumor Volume (mm3)'],
                             'Standard deviation':general_stats['Tumor Volume (mm3)']['std'],
                             'SEM':sem_stats['Tumor Volume (mm3)']
                                    })

left_align(sem_stats_df.head(10))



,Mean,Median,Variance,Standard deviation,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.322552,50.854632,42.351070,6.507770,0.512884
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [ ]:
#Generate a bar plot using both Pandas's DataFrame.plot()
# figtitle = plt.figure('Study Treatment Regimen')
# figtitle.canvas.set_window_title('Study Treatment Regimen')
merge_stats['Tumor Volume (mm3)']['count'].plot(kind='bar', color='r', figsize=(8,5), width=0.5)
plt.xlim(-0.5, len(merge_stats)-0.5)
plt.ylim(0, max(merge_stats['Tumor Volume (mm3)']['count'])+5)

# count_chart.set_xlabel('Drug Regimen')
# count_chart.set_ylabel('Total Measurements Taken')

#Labels
plt.title('Study Treatment Regimen')
plt.xlabel('Drugs')
plt.ylabel('Total Measurements Taken')
# figtitle = plt.figure('Study Treatment Regimen')
# figtitle.canvas.set_window_title('Study Treatment Regimen')

plt.figure(1)
#plt.show()
plt.tight_layout()


# #Generate a bar plot using Matplotlib's pyplot

x_axis = np.arange(len(merge_stats))
tick_locations = [value for value in x_axis]
plt.figure(figsize=(8,5))

plt.bar(x_axis, merge_stats['Tumor Volume (mm3)']['count'], color='r', align="center", width=0.5)
plt.xticks(tick_locations, merge_stats.index, rotation="vertical")

# Set x and y limits
plt.xlim(-0.5, len(x_axis)-0.5)
plt.ylim(0, max(merge_stats['Tumor Volume (mm3)']['count'])+5)

#Labels
plt.title("Study Treatment Regimen")
plt.xlabel("Drugs")
plt.ylabel("Total Measurements Taken")

plt.figure(2)
plt.tight_layout()
plt.show()

In [ ]:
# #Generate a bar plot using Matplotlib's pyplot

# x_axis = np.arange(len(merge_stats))
# tick_locations = [value for value in x_axis]
# plt.figure(figsize=(8,5))

# plt.bar(x_axis, merge_stats['Tumor Volume (mm3)']['count'], color='r', align="center", width=0.5)
# plt.xticks(tick_locations, merge_stats.index, rotation="vertical")

# # Set x and y limits
# plt.xlim(-0.5, len(x_axis)-0.5)
# plt.ylim(0, max(merge_stats['Tumor Volume (mm3)']['count'])+5)

# #Labels
# plt.title("Study Treatment Regimen")
# plt.xlabel("Drugs")
# plt.ylabel("Total Measurements Taken")

# plt.figure(2)
# plt.show()
# plt.tight_layout()


In [ ]:
#Generate a pie plot using Pandas's DataFrame.plot() that shows the distribution of female or male mice in the study.

mice_gender_df = merge_df.drop_duplicates(subset=['Mouse ID'],keep='first')
mice_gender_df = mice_gender_df.groupby(['Sex']).count()
mice_gender_df['Mouse ID'].plot(kind='pie', explode=(0.1, 0), colors=['pink', 'lightblue'], shadow=True, 
                                autopct="%1.1f%%", startangle = 145
                                )
plt.title("Mice Gender Distribution in study")
plt.ylabel('')
plt.figure()
plt.tight_layout()



In [ ]:
# #Generate a pie plot using Matplotlib's pyplot that shows the distribution of female or male mice in the study.
# # mice_gender_df = merge_df.drop_duplicates(subset=['Mouse ID'],keep='first')
# # mice_gender_df = mice_gender_df.groupby(['Sex']).count()

#Pie Chart
plt.pie(mice_gender_df['Mouse ID'], explode=(0.1, 0), labels=mice_gender_df.index, 
        colors=['pink', 'lightblue'], shadow=True, autopct="%1.1f%%", startangle = 145)
plt.title("Mice Gender Distribution in study")

plt.figure(4)
plt.tight_layout()

In [ ]:
#Creating new Data Frame with the most promising treatment regimens: #Capomulin, Ramicane, Infubinol, and Ceftamin. 
promising_treatments = clean_merge[(clean_merge['Drug Regimen'] == 'Capomulin') |
                                   (clean_merge['Drug Regimen'] == 'Ramicane')  | 
                                   (clean_merge['Drug Regimen'] == 'Infubinol') | 
                                   (clean_merge['Drug Regimen'] == 'Ceftamin')
                                  ]
#Keep final tumor volume of each mouse by dropping duplicates and keep last since data it's sorted for Tumor Volume(mm3)
promising_treatments = promising_treatments.drop_duplicates(subset=['Mouse ID'],keep='last')

#Creating a variable for each drug treatment
tumor_capomulin = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Capomulin', :]
tumor_ramicane = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Ramicane', :]
tumor_infubinol = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Infubinol', :]
tumor_ceftamin = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Ceftamin', :]

#Generate whisker plots per Drug Treatment
red_diamond = dict(markerfacecolor='r', marker='D')
fig1, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, figsize=(5,15))
box_capomulin = tumor_capomulin['Tumor Volume (mm3)']
ax1.set_title('Capomulin Treatment')
ax1.set_ylabel('Tumor Volume (mm3)')
ax1.boxplot(box_capomulin, flierprops=red_diamond)

box_ramicane = tumor_ramicane['Tumor Volume (mm3)']
ax2.set_title('Ramicane Treatment')
ax2.set_ylabel('Tumor Volume (mm3)')
ax2.boxplot(box_ramicane, flierprops=red_diamond)

box_infubinol = tumor_infubinol['Tumor Volume (mm3)']
ax3.set_title('Infubinol Treatment')
ax3.set_ylabel('Tumor Volume (mm3)')
ax3.boxplot(box_infubinol, flierprops=red_diamond)

box_ceftamin = tumor_ceftamin['Tumor Volume (mm3)']
ax4.set_title('Ceftamin Treatment')
ax4.set_ylabel('Tumor Volume (mm3)')
ax4.boxplot(box_ceftamin, flierprops=red_diamond)

plt.show()

#Calculate outliers
#-------------------------Capomulin-------------------------#
quartiles_capomulin = box_capomulin.quantile([.25, .75])
lowerq_capomulin = quartiles_capomulin[0.25]
upperq_capomulin = quartiles_capomulin[0.75]
iqr_capomulin = upperq_capomulin-lowerq_capomulin
lower_bound_capomulin = lowerq_capomulin - (1.5*iqr_capomulin)
upper_bound_capomulin = upperq_capomulin + (1.5*iqr_capomulin)

#-------------------------Ramicane-------------------------#
quartiles_ramicane = box_ramicane.quantile([.25, .75])
lowerq_ramicane = quartiles_ramicane[0.25]
upperq_ramicane = quartiles_ramicane[0.75]
iqr_ramicane = upperq_ramicane-lowerq_ramicane
lower_bound_ramicane = lowerq_ramicane - (1.5*iqr_ramicane)
upper_bound_ramicane = upperq_ramicane + (1.5*iqr_ramicane)

#-------------------------Infubinol-------------------------#
quartiles_infubinol = box_infubinol.quantile([.25, .75])
lowerq_infubinol = quartiles_infubinol[0.25]
upperq_infubinol = quartiles_infubinol[0.75]
iqr_infubinol = upperq_infubinol-lowerq_infubinol
lower_bound_infubinol = lowerq_infubinol - (1.5*iqr_infubinol)
upper_bound_infubinol = upperq_infubinol + (1.5*iqr_infubinol)

#-------------------------Ceftamin-------------------------#
quartiles_ceftamin = box_ceftamin.quantile([.25, .75])
lowerq_ceftamin = quartiles_ceftamin[0.25]
upperq_ceftamin = quartiles_ceftamin[0.75]
iqr_ceftamin = upperq_ceftamin-lowerq_ceftamin
lower_bound_ceftamin = lowerq_ceftamin - (1.5*iqr_ceftamin)
upper_bound_ceftamin = upperq_ceftamin + (1.5*iqr_ceftamin)

#Create Data Frame with outliers per Drug Treatments
treatment_regimen_df = pd.DataFrame({'Drug Treatments':['Capomulin', 'Ceftamin', 'Infubinol', 'Ramicane'],
                                     'lowerq':[lowerq_capomulin, lowerq_ceftamin, lowerq_infubinol, lowerq_ramicane], 
                                     'upperq':[upperq_capomulin, upperq_ceftamin, upperq_infubinol, upperq_ramicane], 
                                     'iqr':[iqr_capomulin, iqr_ceftamin, iqr_infubinol, iqr_ramicane],
                                     'lbound *':[lower_bound_capomulin, lower_bound_ceftamin, lower_bound_infubinol, lower_bound_ramicane],
                                     'ubound **':[upper_bound_capomulin, upper_bound_ceftamin, upper_bound_infubinol, upper_bound_ramicane]
                                    })
                                 

print('*Values below could be outliers')
print('**Values above could be outliers')
left_align(treatment_regimen_df)


In [ ]:
#Line Plot of Tumor Volume vs Timepoint of a mouse treated with Capomulin

######################################################################################
promising_treatments = clean_merge[(clean_merge['Drug Regimen'] == 'Capomulin') |
                                   (clean_merge['Drug Regimen'] == 'Ramicane')  | 
                                   (clean_merge['Drug Regimen'] == 'Infubinol') | 
                                   (clean_merge['Drug Regimen'] == 'Ceftamin')
                                  ]
####################################################################################### Needs to be removed once plotting issue resolved
linedata = promising_treatments[(promising_treatments['Drug Regimen']=='Capomulin') & 
                               (promising_treatments['Mouse ID']=='s185')
                               ] 
#linedata = linedata[linedata['Mouse ID']=='s185']

# Plot the line
plt.plot(linedata['Timepoint'], linedata['Tumor Volume (mm3)'])
plt.title('Capomulin Tumor Volume vs Timepoint (s185)')
plt.xlabel('Timepoint')
plt.ylabel('Tumor Volume (mm3)')
plt.show()

#linedata.head(10)

In [ ]:
#Scatter Plot of Mouse Weight vs Avg Tumor Volume for the Capomulin treatment regimen

######################################################################################
promising_treatments = clean_merge[(clean_merge['Drug Regimen'] == 'Capomulin') |
                                   (clean_merge['Drug Regimen'] == 'Ramicane')  | 
                                   (clean_merge['Drug Regimen'] == 'Infubinol') | 
                                   (clean_merge['Drug Regimen'] == 'Ceftamin')
                                  ]

#Keep final tumor volume of each mouse by dropping duplicates and keep last since data it's sorted for Tumor Volume(mm3)
promising_treatments = promising_treatments.drop_duplicates(subset=['Mouse ID'],keep='last')

#Creating a variable for each drug treatment
tumor_capomulin = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Capomulin', :]
tumor_ramicane = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Ramicane', :]
tumor_infubinol = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Infubinol', :]
tumor_ceftamin = promising_treatments.loc[promising_treatments['Drug Regimen'] == 'Ceftamin', :]

####################################################################################### Needs to be removed once plotting issue resolved
#group_promising_treatments = promising_treatments.groupby(['Mouse ID']).mean()
#scatterdata = group_promising_treatments[(promising_treatments['Drug Regimen']=='Capomulin')]
#scatterdata = promising_treatments[(promising_treatments['Drug Regimen']=='Capomulin')]
group_tumor_capomulin = tumor_capomulin.groupby(['Mouse ID', 'Drug Regimen']).mean()

avg_tumorvolume = group_tumor_capomulin['Tumor Volume (mm3)']
mouse_weight = group_tumor_capomulin['Weight (g)']        
plt.scatter(mouse_weight, avg_tumorvolume, marker="o", facecolors="red", edgecolors="black")

#Correlation efficient calculation and Plot linear regression model
x_values = mouse_weight
y_values = avg_tumorvolume
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Mouse Weight')
plt.ylabel('Avg Tumor Volume')
print(f"The r-squared is: {rvalue**2}")
plt.tight_layout()